# 词的相似性和类比任务


In [1]:
import os
import torch
from torch import nn
from d2l import torch as d2l

## 加载预训练词向量

以下列出维度为50、100和300的预训练GloVe嵌入，可从[GloVe网站](https://nlp.stanford.edu/projects/glove/)下载。预训练的fastText嵌入有多种语言。这里我们使用可以从[fastText网站](https://fasttext.cc/)下载300维度的英文版本（“wiki.en”）。


In [2]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

**为什么需要双向映射？**
| 属性                 | 类型 | 用途                |
| ------------------ | -- | ----------------- |
| **`idx_to_token`** | 列表 | **索引转单词**（用于展示结果） |
| **`token_to_idx`** | 字典 | **单词转索引**（用于查询向量） |

**完整数据流动**
```Python
# 实例化
glove = TokenEmbedding('glove.6B.50d')

# 内部结构
glove.idx_to_token  # → ['<unk>', 'the', 'of', 'and', ...]
glove.idx_to_vec    # → tensor([[0.0, 0.0, ...], [0.046, 0.213, ...], ...])
glove.unknown_idx   # → 0
glove.token_to_idx  # → {'<unk>': 0, 'the': 1, 'of': 2, ...}
```

| 代码行                         | 功能      | 关键技术        |
| --------------------------- | ------- | ----------- |
| `self._load_embedding(...)` | 加载预训练向量 | 自动下载 + 文本解析 |
| `self.unknown_idx = 0`      | OOV词处理  | 默认索引机制      |
| `字典推导式`                     | 构建反向映射  | O(1)查询效率    |


**每个部分的含义**

`len(idx_to_vec[0])  ：获取词向量的维度（如 50, 100, 300）`
```Python
idx_to_vec[0]  # → [0.04656, 0.21318, ...]
len(idx_to_vec[0])  # → 50（假设是50维）
[0] * len(idx_to_vec[0])  ：创建一个全零列表
```
```Python
[0] * 50  # → [0, 0, 0, ..., 0]（50个0）
[[0] * len(idx_to_vec[0])]  ：包装成二维列表（单个向量）
```
```Python
[[0, 0, 0, ..., 0]]  # → 形状: (1, 50)
+ idx_to_vec  ：列表拼接，将全零向量放在最前面
```
```Python
[[0,0,...,0]] + [[0.04656,...], [-0.071549,...], ...]
# → [[0,0,...,0], [0.04656,...], [-0.071549,...], ...]
```

1. 将单词转换为索引
```Python
indices = [self.token_to_idx.get(token, self.unknown_idx)
           for token in tokens]
```
**列表推导式分解：**

外层：遍历 `tokens` 列表中的每个 `token`
内层：`self.token_to_idx.get(token, self.unknown_idx)`

**dict.get(key, default)的作用：**
```python
# 如果 token 在字典中
self.token_to_token['chip']  # → 返回对应的索引，如 305

# 如果 token 不在字典中
self.token_to_token['unknownword']  # → 返回 self.unknown_idx (0)
避免 KeyError：即使查询未知词，也不会报错，而是返回 <unk> 的索引。
```
2. 从张量中提取向量
```Python
vecs = self.idx_to_vec[torch.tensor(indices)]
```
**执行过程：**
```Python
# 假设 indices = [305, 102, 0] (chip, cpu, <unk>)
idx_tensor = torch.tensor(indices)  # → tensor([305, 102, 0])

# PyTorch 高级索引：一次性提取多行
vecs = self.idx_to_vec[idx_tensor]
# → 返回形状为 (3, embed_dim) 的张量
```
**索引机制：**

`self.idx_to_vec` 形状：(词表大小, 向量维度)

`torch.tensor(indices)` 形状：(查询词数量,)

结果：返回 [305, 102, 0] 行对应的向量，形状 (查询词数量, 向量维度)

3. 返回结果
```Python
return vecs
```
返回值：torch.Tensor，可直接用于神经网络

In [3]:
#@save
class TokenEmbedding:
    """GloVe嵌入"""
    '''
    embedding_name：预训练嵌入名称（如'glove.6B.100d'）
    idx_to_token：索引→单词的列表（如['<unk>','the','of',...]）
    idx_to_vec：索引→向量的张量（形状：(词表大小,嵌入维度)）
    unknown_idx=0：未知词默认映射到索引0（<unk>）
    token_to_idx：单词→索引的字典，用于快速查询
    '''
    def __init__(self, embedding_name):
        '''
        1. 加载预训练嵌入
        调用：_load_embedding()方法自动下载并解析词向量文件
        返回：两个对象
            idx_to_token：列表，索引→单词（如 ['<unk>','the','of',...]）
            idx_to_vec：张量，索引→向量（形状 [词表大小,嵌入维度]）
        '''
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        '''
        2. 设置未知词索引
        作用：为未登录词（OOV）指定默认索引
        约定：索引0通常为特殊词<unk>（Unknown）
        意义：查询不存在单词时返回该索引
        '''
        self.unknown_idx = 0
        '''
        3. 构建反向映射字典
        字典推导式：将idx_to_token列表反转
        结果：token_to_idx是单词→索引的字典（如{'the':1,'of':2}）
        目的：实现O(1)时间复杂度的单词快速查找
        '''
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}
    # 加载嵌入文件
    def _load_embedding(self, embedding_name):
        '''
        1. 初始化容器
        idx_to_token=['<unk>']：预先放入未知词符号，确保索引0是<unk>
        idx_to_vec=[]：向量列表暂时为空（后续动态填充）
        '''
        idx_to_token, idx_to_vec = ['<unk>'], []
        # 2. 下载并解压文件
        data_dir = d2l.download_extract(embedding_name)
        # GloVe网站：https://nlp.stanford.edu/projects/glove/
        # fastText网站：https://fasttext.cc/
        # 3. 打开向量文件
        # 文件路径：{data_dir}/vec.txt；文件格式：纯文本，每行是一个词+向量
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            '''
            4. 逐行解析
            line.rstrip().split(' ')：移除换行符并空格分割
            elems[0]：第一个元素是单词（如'the'）
            elems[1:]：剩余元素是向量维度（字符串列表）
            [float(elem)...]：转换为浮点数列表
            '''
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                '''
                5. 跳过标题行
                目的：跳过文件开头的元信息行（如fastText的首行）
                判断：有效向量行至少有1个维度，len(elems)>1确保不是空行或标题
                追加：将词和向量分别加入对应列表
                '''
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        '''
        6. 插入未知词向量
        操作：在索引0处插入全零向量作为<unk>的嵌入
        维度：len(idx_to_vec[0])获取第一个向量的长度（如50、100、300）
        结果：idx_to_vec[0]是全零向量，对应idx_to_token[0]='<unk>'
        '''
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        '''
        7. 转换为PyTorch张量
        torch.tensor(idx_to_vec)：将嵌套列表转为PyTorch张量
        形状：(词表大小,嵌入维度)
        优点：可直接用于神经网络，支持GPU加速
        '''
        return idx_to_token, torch.tensor(idx_to_vec)
    # 索引访问
    def __getitem__(self, tokens):
        '''
        self.token_to_idx ：单词→索引的字典（如 {'chip':305,'cpu':102}）
        .get(key,default)：字典的安全查询方法
            如果key存在，返回对应值
            如果key不存在，返回default（避免KeyError）
        '''
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs
    # 获取词表大小
    def __len__(self):
        return len(self.idx_to_token)

下面我们加载50维GloVe嵌入（在维基百科的子集上预训练）。创建`TokenEmbedding`实例时，如果尚未下载指定的嵌入文件，则必须下载该文件。


In [4]:
glove_6b50d = TokenEmbedding('glove.6b.50d')

输出词表大小。词表包含400000个词（词元）和一个特殊的未知词元。


In [5]:
len(glove_6b50d)

400001

我们可以得到词表中一个单词的索引，反之亦然。


In [6]:
'''
token_to_idx：单词→索引的字典
作用：查询单词'beautiful'在词汇表中的索引
返回值：整数（如3367）
idx_to_token：索引→单词的列表
作用：查询索引3367对应的单词
返回值：字符串（如'beautiful'）
'''
glove_6b50d.token_to_idx['beautiful'], 
glove_6b50d.idx_to_token[3367]

'beautiful'

## 应用预训练词向量

使用加载的GloVe向量，我们将通过下面的词相似性和类比任务中来展示词向量的语义。

### 词相似度

与`subsec_apply-word-embed`类似，为了根据词向量之间的余弦相似性为输入词查找语义相似的词，我们实现了以下`knn`（$k$近邻）函数。


In [7]:
'''
W：词向量矩阵，形状(词表大小,向量维度)
x：查询向量，形状(向量维度,)或(1,向量维度)
k：返回最相似的k个词
topk：最相似词的索引列表
[cos[int(i)] for i in topk]：对应的余弦相似度分数列表
'''
def knn(W, x, k):

    '''

    1. 计算余弦相似度
    增加1e-9以获得数值稳定性
    分子：点积
        x.reshape(-1,)：将查询向量展平为一维（如(300,)→(300,)）
        torch.mv(W,x)：矩阵-向量乘法，计算每个词向量与查询向量的点积
        结果：形状 (词表大小,)，每个元素是Wi⋅x 
    分母：模长乘积
        torch.sum(W*W, axis=1)：每个词向量的模长平方（逐元素平方后求和）
        +1e-9：数值稳定性，防止除零错误
        torch.sqrt(...)：词向量的模长||Wi||
        (x*x).sum()：查询向量的模长平方||x||^2
        torch.sqrt(...)：查询向量的模长||x|| 
        整体：||Wi||⋅||x||
        结果：cos 是形状为(词表大小,)的张量，元素范围[-1,1]
    '''
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum()))
    '''
    2. 返回最相似的k个词
    torch.topk(cos,k=k) ：返回cos中最大的k个值及其索引
    _：忽略值（相似度分数，后面会重新提取）
    topk：索引张量，形状(k,)，表示最相似的k个词的位置
    '''
    _, topk = torch.topk(cos, k=k)
    '''
    3. 提取相似度分数
    topk：索引列表（如tensor([3367,102,89])）
    列表推导式：遍历每个索引i，从cos中提取对应分数
    int(i)：将张量元素转为Python整数（用于索引）
    结果：返回(索引列表,分数列表)元组
    '''
    return topk, [cos[int(i)] for i in topk]

然后，我们使用`TokenEmbedding`的实例`embed`中预训练好的词向量来搜索相似的词。


In [8]:
'''
query_token：查询词（如'beautiful'）
k：返回最相似的词数量
embed：TokenEmbedding实例（包含词向量）
'''
def get_similar_tokens(query_token, k, embed):
    '''
    1. 查找最相似的 k+1 个词
    为什么k+1？
    因为查询词本身的相似度最高（cosine=1.0），必然排在第一位。我们需要排除自己，所以查找k+1个，然后跳过第一个
    参数详解：
    embed.idx_to_vec：所有词的向量矩阵，形状(词表大小,向量维度)
    embed[[query_token]]：查询词的向量，形状(1,向量维度)
    k+1：返回数量（包含查询词本身）
    返回结果：
        topk：索引张量，如[query_idx,sim_idx1,sim_idx2,...]
        cos：相似度列表，如[1.0,0.89,0.85,...]
    '''
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    '''
    2. 跳过查询词，遍历结果
    topk[1:]：从第2个元素开始（排除查询词的索引）
    cos[1:]：从第2个元素开始（排除查询词的相似度1.0）
    zip(...,...)：将索引和相似度配对遍历
    第1次循环：i=topk[1],c=cos[1]
    第2次循环：i=topk[2],c=cos[2]
    '''
    for i, c in zip(topk[1:], cos[1:]):  # 排除输入词
        '''
        embed.idx_to_token[int(i)]：将索引转换为单词
        float(c)：将张量转为浮点数
        :.3f：格式化输出，保留3位小数
        '''

        print(f'{embed.idx_to_token[int(i)]}：cosine相似度={float(c):.3f}')

**执行流程**

**第一步：获取查询词向量**
```Python
embed[['chip']]  # glove_6b50d[['chip']]
```
- 在 glove_6b50d 中查找 'chip' 的词向量
- 结果：形状 (1, 50) 的张量

**第二步：执行K近邻搜索（查找4个）**
```Python
knn(embed.idx_to_vec, embed[['chip']], k=4)  # 注意：k+1=4
```
- 计算 'chip' 与词表中所有词的余弦相似度
- 返回：最相似的 4 个词的索引和相似度
**为什么查找4个？**

因为 'chip' 本身的相似度是 1.0，必然排在第一位。需要排除自己，所以查找 3+1=4 个。

**第三步：处理结果并打印**
```Python
for i, c in zip(topk[1:], cos[1:]):  # 跳过第一个（查询词自己）
    print(f'{embed.idx_to_token[int(i)]}：cosine相似度={float(c):.3f}')
```
**预期输出示例**

基于语义相关性，可能的输出如下：
```python
processor：cosine相似度=0.756
memory：cosine相似度=0.732
cpu：cosine相似度=0.718
```
**为什么是这些词？**
- 'chip' 在语义上与计算机硬件相关
- 'processor'（处理器）、'memory'（内存）、'cpu'（中央处理器）都与芯片紧密相关

**不同词向量的结果差异**

**如果使用 glove_6b50d（50维）**
- 结果可能偏向通用语义
- 相似度分数相对较低（0.6-0.8）

**如果使用 glove_6b100d（100维）**
```Python
get_similar_tokens('chip', 3, glove_6b100d)
# 可能输出：
# microprocessor：cosine相似度=0.812
# semiconductor：cosine相似度=0.798
# cpu：cosine相似度=0.785
```
- 更高维度的词向量语义更精确
- 可能出现更专业的词（如 microprocessor, semiconductor）

**无效查询的处理**

**查询不在词表中的词**
```Python
get_similar_tokens('nonexistentword', 3, glove_6b50d)
```
**输出：**
```python
<unk>：cosine相似度=0.000
<unk>：cosine相似度=0.000
<unk>：cosine相似度=0.000
```
因为未知词被映射到 <unk>，其向量是全零向量，与所有词的相似度都是0。

**查询特殊词**
```Python
get_similar_tokens('<unk>', 3, glove_6b50d)
# 输出：<unk>相关的词（通常是低频词）
```
**调试技巧**

**检查 'chip' 是否在词表中**
```Python
if 'chip' in glove_6b50d.token_to_idx:
    print(f"'chip' 的索引是: {glove_6b50d.token_to_idx['chip']}")
else:
    print("'chip' 不在词表中，将被映射到 <unk>")
 ```
**查看 'chip' 的向量**
```Python
chip_vec = glove_6b50d[['chip']]
print(chip_vec.shape)  # → torch.Size([1, 50])
print(chip_vec[:5])    # 打印前5维
```
**手动验证结果**
```Python
# 获取最相似的词并检查是否真的相似
topk, cos = knn(glove_6b50d.idx_to_vec, glove_6b50d[['chip']], 4)
for i, c in zip(topk[1:], cos[1:]):
    word = glove_6b50d.idx_to_token[int(i)]
    print(f"{word}: {float(c):.3f}")

# 检查第一个是否是 'chip' 自己
first_word = glove_6b50d.idx_to_token[int(topk[0])]
print(f"第一个词是: {first_word} (应该是 'chip' 自己)")
```

In [9]:
'''
'chip'：查询词（芯片）
3：返回3个最相似的词
glove_6b50d：GloVe 6B 50维词向量实例
'''
get_similar_tokens('chip', 3, glove_6b50d)

chips：cosine相似度=0.856
intel：cosine相似度=0.749
electronics：cosine相似度=0.749


下面输出与“baby”和“beautiful”相似的词。


In [10]:
get_similar_tokens('baby', 3, glove_6b50d)

babies：cosine相似度=0.839
boy：cosine相似度=0.800
girl：cosine相似度=0.792


In [11]:
get_similar_tokens('beautiful', 3, glove_6b50d)

lovely：cosine相似度=0.921
gorgeous：cosine相似度=0.893
wonderful：cosine相似度=0.830


### 词类比

除了找到相似的词，我们还可以将词向量应用到词类比任务中。
例如，“man” : “woman” :: “son” : “daughter”是一个词的类比。
“man”是对“woman”的类比，“son”是对“daughter”的类比。
具体来说，词类比任务可以定义为：
对于单词类比$a : b :: c : d$，给出前三个词$a$、$b$和$c$，找到$d$。
用$\text{vec}(w)$表示词$w$的向量，
为了完成这个类比，我们将找到一个词，
其向量与$\text{vec}(c)+\text{vec}(b)-\text{vec}(a)$的结果最相似。


idx_to_vec 是 TokenEmbedding 类中的核心属性，是一个 PyTorch 张量，存储了词汇表中所有单词的词向量。

**基本定义**
```Python
self.idx_to_vec  # 形状: (词表大小, 嵌入维度)
```
它是一个二维张量，每行对应一个单词的向量表示。

**数据结构**

**形状示例**
```Python
# 如果加载 GloVe 6B 50d
idx_to_vec.shape  # → torch.Size([400001, 50])

# 如果加载 GloVe 6B 100d
idx_to_vec.shape  # → torch.Size([400001, 100])
```
**维度含义**
- 第0维（400001）：词汇表大小（包含 <unk>）
- 第1维（50或100）：词向量的维度

**索引映射关系**

idx_to_vec 与 idx_to_token 严格一一对应：

| 索引   | `idx_to_token[i]` | `idx_to_vec[i]`            | 说明       |
| ---- | ----------------- | -------------------------- | -------- |
| 0    | `'<unk>'`         | `[0.0, 0.0, ..., 0.0]`     | 未知词，全零向量 |
| 1    | `'the'`           | `[0.04656, 0.21318, ...]`  | 高频词      |
| 2    | `','`             | `[0.12345, -0.06789, ...]` | 标点符号     |
| 3    | `'.'`             | `[0.98765, 0.05432, ...]`  | 标点符号     |
| ...  | ...               | ...                        | ...      |
| 3367 | `'beautiful'`     | `[0.23456, -0.12345, ...]` | 具体单词     |

**内容示例**
```Python
# 查看前5个词向量
glove_6b50d.idx_to_vec[:5]
# tensor([[ 0.0000,  0.0000,  ...,  0.0000],  # <unk>
#         [ 0.0466,  0.2132,  ...,  0.0539],  # the
#         [ 0.1234, -0.0679,  ...,  0.0456],  # ,
#         [ 0.9877,  0.0543,  ...,  0.0234],  # .
#         [ ... ]])                           # of
```

**为什么有效？（理论基础）**

1. 分布式假设<br>
"一个词的含义由它周围的词决定"。词向量训练时，语义相关的词在向量空间中 彼此接近。

2. 线性子空间假设（Mikolov et al. 2013）<br>
词向量空间中的语义关系是近似线性的：
- "性别"关系：v(king) - v(man) ≈ v(queen) - v(woman)
- "首都"关系：v(Paris) - v(France) ≈ v(Berlin) - v(Germany)

3. 向量平移<br>
v(king) - v(man) 是一个从"man"到"king"的平移向量

将这个平移应用到 v(woman) 上，得到 v(queen)

**几何可视化**
```python
      v(king) ≈ v(man) + (v(queen) - v(woman))

          v(king)      v(queen)
              *        *
              |        |
              |        |
              *        *
          v(man)      v(woman)

          [王室+男性]   [王室+女性]
```
向量差 v(king) - v(man) ≈ 向量差 v(queen) - v(woman)

**其他类比示例**

**首都关系**
```Python
# Paris : France :: Berlin : Germany
x = v(France) - v(Paris) + v(Berlin)  # 应接近 v(Germany)
```
**复数关系**
```Python
# mouse : mice :: child : children
x = v(mice) - v(mouse) + v(child)  # 应接近 v(children)
```
**反义词关系**
```python
# big : small :: hot : cold
x = v(small) - v(big) + v(hot)  # 应接近 v(cold)
```
**线性代数的本质**

这个公式的本质是在向量空间中进行线性组合：
- 向量减法 ：提取 语义差异（如"王室"属性）
- 向量加法：组合语义（"王室" + "女性"）
**目标：**找到满足 v(d) ≈ x 的词 d，即最近邻搜索。

**局限性**

虽然这个公式在很多情况下有效，但不是绝对可靠：
- 多义性：bank 有"银行"和"河岸"两个意思，向量是混合表示
- 文化偏差：词向量可能继承训练数据中的偏见（如性别、种族）
- 非线性关系：某些语义关系无法被线性捕捉


In [12]:
def get_analogy(token_a, token_b, token_c, embed):
    '''
    第一步：获取三个词向量
    输入：三个词（如'man','king','woman'）
    输出：形状为(3,向量维度)的张量
    索引：
        vecs[0]→v(token_a)（如v(man)）
        vecs[1]→v(token_b)（如v(king)）
        vecs[2]→v(token_c)（如v(woman）
    '''
    vecs = embed[[token_a, token_b, token_c]]
    '''
    第二步：执行向量运算
    数学表达：x=v(king)-v(man)+v(woman)
    语义解释：
        v(king)-v(man)：去除"男性"属性，保留"王室"属性
        +v(woman)：添加"女性"属性
        结果：应该接近v(queen)
    '''
    x = vecs[1] - vecs[0] + vecs[2]
    '''
    第三步：寻找最相似的词
    在词表中搜索与向量x最相似的1个词
    topk：最相似词的索引（如queen的索引）
    cos：相似度分数（应接近1.0）
    '''
    topk, cos = knn(embed.idx_to_vec, x, 1)
    '''
    第四步：返回结果词
    将张量索引转为Python整数
    通过idx_to_token将索引转为单词
    返回：答案词（如'queen'）
    '''
    return embed.idx_to_token[int(topk[0])]  # 删除未知词

让我们使用加载的词向量来验证“male-female”类比。


**执行过程详解**

**第一步：获取三个词向量**
```Python
vecs = glove_6b50d[['man', 'woman', 'son']]
# vecs[0] = v('man')
# vecs[1] = v('woman')
# vecs[2] = v('son')
```

**第二步：执行向量运算**
```Python
x = vecs[1] - vecs[0] + vecs[2]
# x = v('woman') - v('man') + v('son')
```

**语义解释：**
- v('woman') - v('man') ：提取"女性"相对于"男性"的 语义差向量
- **+ v('son')**：将该差向量应用到"儿子"上
- 结果 x：应接近  **v('daughter')**

**第三步：寻找最相似词**
```Python
topk, cos = knn(glove_6b50d.idx_to_vec, x, 1)
```
- 在词表中搜索与向量 x 最接近的词
- topk[0]：答案词的索引
- cos[0]  ：相似度分数（应接近1.0）

**第四步：返回结果**
```Python
return glove_6b50d.idx_to_token[int(topk[0])]
```
- 将索引转换为单词字符串

**预期输出**
```Python
result = get_analogy('man', 'woman', 'son', glove_6b50d)
print(result)  # → 'daughter'
```
**可能的结果**

**理想情况（词向量质量高）**：'daughter'

**次优情况（50维可能不够精确）**
```python
# 可能返回语义相关的词：
'girl'        # 女性孩子
'daughter.'   # 带标点的变体
'child'       # 泛化过度
```
**失败情况（输入词不在词表中）**
```python
# 如果 'son' 不在词表中，会被映射为 <unk>
# 结果可能是无意义的词
```
**验证与调试**

**检查输入词是否存在**
```Python
for token in ['man', 'woman', 'son']:
    if token not in glove_6b50d.token_to_idx:
        print(f"'{token}' 不在词表中！")
# 确保都返回 False
```
**手动执行向量运算**
```Python
# 获取向量
v_man = glove_6b50d[['man']]
v_woman = glove_6b50d[['woman']]
v_son = glove_6b50d[['son']]

# 执行运算
x = v_woman - v_man + v_son

# 查看最近邻
topk, cos = knn(glove_6b50d.idx_to_vec, x, 5)
for i, c in zip(topk, cos):
    print(f"{glove_6b50d.idx_to_token[int(i)]}: {float(c):.3f}")
```
**期望输出前五名：**
```python
daughter: 0.892  # 第1名
girl: 0.845
child: 0.812
son: 0.798       # 输入词本身可能也接近
boy: 0.765
```
**局限性**

1. 维度限制

50维 GloVe 可能无法完美捕捉复杂关系，300维版本效果更好。

2. 词表覆盖

如果 'man', 'woman', 'son' 中有词不在词表中，结果会无效。

3. 社会偏见

词向量可能包含训练数据的偏见，如：

```Python
# 经典偏见示例
get_analogy('man', 'doctor', 'woman', glove)
# 可能返回 'nurse'（反映性别刻板印象）
```

In [13]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

下面完成一个“首都-国家”的类比：
“beijing” : “china” :: “tokyo” : “japan”。
这说明了预训练词向量中的语义。


In [14]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

另外，对于“bad” : “worst” :: “big” : “biggest”等“形容词-形容词最高级”的比喻，预训练词向量可以捕捉到句法信息。

In [15]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

为了演示在预训练词向量中捕捉到的过去式概念，我们可以使用“现在式-过去式”的类比来测试句法：“do” : “did” :: “go” : “went”。


In [16]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'